In [9]:
import sys
import numpy as np 
import sympy 
import dataclasses
from typing import Optional

# other files I created 
import header
import linearize
import initialize
import kalmanFilter as kf

### The Dynamics and Measurement Model

The phase will be estimated by the following process: 

> $\phi_{k} = \phi_{0} + 2\pi(f_{d,k}kTs + \frac{1}{2} \dot{f}_{d,k} k^2 Ts^2)$

Wherever we estimate a parameter, we incur a noise term on that estimate. The IMU will be reporting the observer velocity, $v_0$, but will impart a biased noise term, $\eta_{IMU}$, onto it. Accounting for this in the traditional doppler frequency equation yields:  

> ### $\hat{f}_d = \frac{v+v_0+\eta_{IMU}}{v+v_s} + \eta_{clock}$
Where $\hat{f}_d$ is the doppler estimate, $v$ is the speed of sound, $v_s$ is the signal source velocity, and $\eta_{clock}$ is the clock process noise.

This requires an estimate of the user velocity from the IMU. The equation used here will be: 

> $\dot{v}^n = f^n -(\Omega^{n}_{e/n} + 2 \Omega^{n}_{i/e})v^n + g^n$

We will make the assumption of 1D velocity, constant angular position states: 

> $\dot{v}^n = f^n -(2 \Omega^{n}_{i/e})v^n + g^n$
>> biased accelerometer output: $f^n = b + \omega_{IMU}$

Here, $\eta_{clock}$ will be a white, thermal noise source. However, the IMU noise will be biased and will require some dynamic model.  

How to I model $\eta_{IMU}$? 

#### FW difference FOGMP model: 

>$b_{k} = b_{k-1} + k*Ts*\dot{b}_{k-1} = b_{k-1} + \frac{k*Ts}{\tau}(b_{k-1} + \omega) = (1+\frac{k*Ts}{\tau})b_{k-1} + \frac{k*Ts}{\tau}*\omega$
>> ... $b_{k} = (1+\frac{k*Ts}{\tau})b_{k-1} + \frac{k*Ts}{\tau}*\omega_{k-1}$

#### Dynamic model in continuous time 
> $ \begin{pmatrix} \dot{\theta} \\ \dot{f}_d \\ \dot{v} \\ \dot{b} \end{pmatrix} = \begin{pmatrix}  f_d \\ \frac{d}{dt} (\frac{\nu + v}{\nu + v_s}) \\ f + b + \omega - 2\Omega^{2}_{i/e}v + g\\ \frac{1}{\tau}b + \omega \end{pmatrix} $

Where the phase is given by the accuulation of the frequency $f(t)$ and the original phase: 

>$ \theta = \theta_0 + \int^{t}_{t_0} f(t) dt$

$f(t)$ should be the result of the doppler shift of the incoming signal.

Note here that the $\dot{f}_d$ state is redundant. We can directly substitute it in for the $\dot{\theta}$ equation: 

> $ \begin{pmatrix} \dot{\theta} \\ \dot{v} \\ \dot{b} \end{pmatrix} = \begin{pmatrix}  \frac{\nu + v}{\nu + v_s} \\ f + b + \omega - 2\Omega^{2}_{i/e}v + g\\ \frac{1}{\tau}b + \omega \end{pmatrix} $

### Code implementation

In [2]:
# variances
varV = .1   # msmt. noise variance 
vphi = 1  # phase process noise in rads/s
vfd  = 2  # doppler frequency process noise in Hz
vfdd = 1  # doppler rate process noise in Hz 
vbias = 1 # variance of non-time-correlated portion of IMU biasing on estimates  

a, phi, b, phi_0, fd, fddot= sympy.symbols('a phi b phi_0 fd fddot')
vars = list([phi, fd, fddot, b]) # state variables 
Ts = 100  # sample time in Hz
Tc = .02 # coherent integration time in seconds 
k = np.linspace(0, 10, num = int(10/Ts))

In [11]:
# measurements 
# linearize the observation matrix 
H = sympy.Matrix([[a*sympy.sin(phi)],[a*sympy.cos(phi)]])  # observation matrix 
dH, varsa = linearize.linearize(H, vars, (2,len(vars)))
D = sympy.diag(varV**2,varV**2)

# dynamics 
A = sympy.Matrix([[1,Ts,(Ts**2)/2,0],[0,1,Ts,0],[0,0,1,0],[0,0,0,1+Ts/Tc]]) # state transition matrix
nx = A.shape[0]                                    # number of states must match columns of A 
B = sympy.Matrix(np.zeros((A.shape)))                            # control input matrix (sparse)
G = sympy.diag(vphi**2, vfd**2, vfdd**2, vbias**2)             # process noise input matrix
ng = G.shape[0]         # number of noise variables must match the colummns of G 

# store all system information in a dataclass 
sys = header.SymSystem(A, B, G, dH, D) # NOTE that dH is linearized, and the states there are defined by 
                                       # their deviation from some nominal value 

In [5]:
# we're almost ready to pass this to the kalman filter, but we'll need a trajectory to linearize the system with. 
# this is a case-by-case tool, so it will require a redo for each situation we wish to face.
st = 2 # disired simulation time in seconds
traj = np.array(np.zeros((nx,int(st*Ts))))
# note that state vector is: np.transpose([phi, fd, fddot, bias])
phi_nom = np.pi/2 # nominal phase value 
for i in range(0,int(st*Ts),1):
    traj[:,i] = np.transpose([phi_nom, 10, 2, .001])

#### The Kalman Filter 

The Kalman filter will require: 
- The system dynamics (we will use a header.SymSystem type)
- The measurement model (same type as above)
- The system statistics, which includes: 
    - The variances of all states and noise sources 
    - The initial covariance and MAP estimates of the states 
- A trajectory to linearize off of in the case of out non-linear dynamics 

In [32]:
# the Kalman filter args look like this: 
# def KalmanFilter(sys: h.SymSystem, init_args: dict, traj: np.ndarray, stats: dict):
# we will package our information to suit this

###### The init_args ######
# sys is defined above 
initials = np.array([.001, 14, 2, .001, 1, np.pi/2])
initials = np.transpose(initials)
var_list = list([phi, fd, fddot, b, a, phi_0])
feilds = list(["A", "B", "G", "C", "D"]) # form elements of control system
init_args_keys = list(["sys","initials","var_list","fields"])
init_arg_fields = list([sys, initials, var_list, feilds])
init_args = dict.fromkeys(init_args_keys)
i = 0
for key in init_args_keys:
    init_args[key] = init_arg_fields[i]
    i = i + 1

###### The statistics of the system, 'stats' ######
# variances
# state variances 
varSphi = 1 # varince of state phi
varSfd  = 1 # variance of doppler state
varSfdd = .1 # variance of doppler rate state
varSbias = .01 # variance of bias state 
P0 = np.diag([varSphi**2, varSfd**2, varSfdd**2, varSbias**2])
# process noise variances 
W = 1
# measurement noise variances 
V = 1 
# initial states 
phi0 = .001
fd0 = 14 # Hz
fddot0 = 1 # Hz/s
bias0 = .0001 # m/s 
x0    = np.transpose(np.array([phi0, fd0, fddot0, bias0]))
stats_keys = list(["P0", "x0", "W", "V"])
fields = list([P0, x0, W, V])
stats = dict.fromkeys(stats_keys)
i = 0
for keys in stats_keys:
    stats[keys] = fields[i]
    i = i + 1

In [ ]:
# run kalman filter 
kf.KalmanFilter(sys, init_args, traj, stats, Ts, T)